In [2]:
import numpy as np
import keras.backend as K
from machinedesign.viz import grid_of_images_default
from skimage.io import imsave
from keras.models import Model
from skimage.transform import resize
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
from machinedesign.autoencoder.interface import load
from sklearn.neighbors import KDTree
from machinedesign.viz import horiz_merge
%matplotlib inline
import matplotlib.pyplot as plt

Using Theano backend.


In [3]:
dataset = '../../data/digits.npz'
data = np.load(dataset)
X = data['X']
y = data['y'] if 'y' in data else None
X = X.astype(np.float32)
X /= 255.

In [5]:
clf = load('../../discr/digits')
clf_enc = Model(inputs=clf.layers[0].input, outputs=clf.get_layer('p_re_lu_4').output)
#clf_enc = Model(inputs=clf.layers[0].input, outputs=clf.layers[0].output)

In [6]:
folder = 'mnist'
data = np.load('../results/{}/gen/generated.npz'.format(folder))
model = load('../results/{}'.format(folder))

In [137]:
#mnist
inp = model.layers[0].input
hid_layer = model.get_layer('activation_3')
hid = hid_layer.output
out = model.layers[-1].output
enc = K.function([inp], hid)
dec = K.function([hid], out)

In [106]:
nb = 1000
xgen = data['generated'][0:nb]
xtrue = X[0:nb]
ytrue = y[0:nb]
x = np.concatenate((xgen, xtrue), axis=0)

class_colors = [
    'r',
    'b',
    'g',
    'crimson',
    'gold',
    'yellow',
    'maroon',
    'm',
    'c',
    'orange'
]

In [107]:
#proj = TSNE(n_components=2)
proj = PCA(n_components=2)
xp = model.predict(x)
xx = np.concatenate((x, xp), axis=0)

xx = clf_enc.predict(xx)
#xx = enc([xx])
#xx = xx.max(axis=(2, 3))
#xx = xx.reshape((xx.shape[0], -1))

hh = proj.fit_transform(xx)
h = hh[0:len(x)]
v = hh[len(x):] - hh[0:len(x)]

In [108]:
colors = ['gray'] * nb + [class_colors[yi] for yi in ytrue]
classes = np.array([-1] * nb + ytrue.tolist())

In [ ]:
fig = plt.figure(figsize=(40, 30))
plt.quiver(h[:, 0], h[:, 1], v[:, 0], v[:, 1], scale=250, color='gray', alpha=1.0)
plt.scatter(h[classes==-1, 0], h[classes==-1, 1], c='lightgray', s=50, label='Generated', marker='o', alpha=0.8)


for cl in range(10):
    plt.scatter(h[classes==cl, 0], h[classes==cl, 1], 
                marker='o', c=class_colors[cl], s=50, label='Digit {}'.format(cl))
plt.legend()
plt.show()